In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import sys

In [2]:
import os

In [3]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

In [11]:
convert("./data/train-images.idx3-ubyte", "./data/train-labels.idx1-ubyte",
        "data/mnist_train.csv", 60000)

In [27]:
df = pd.read_csv("./data/mnist_train.csv", header=None)

In [28]:
df.shape

(60000, 785)

In [29]:
training = df.as_matrix()

In [30]:
Y_train = training[:, 0].astype('int')

In [31]:
X_train = training[:, 1:].astype('float64')

In [32]:
X_train.shape

(60000, 784)

In [34]:
convert("./data/t10k-images.idx3-ubyte", "./data/t10k-labels.idx1-ubyte",
        "./data/mnist_test.csv", 10000)

In [35]:
df = pd.read_csv("./data/mnist_test.csv", header=None)

In [36]:
test = df.as_matrix()

In [37]:
test.shape

(10000, 785)

In [38]:
Y_test = test[:, 0].astype('int')

In [39]:
X_test = test[:, 1:].astype('float64')

In [40]:
np.min(X_test)

0.0

In [41]:
np.max(X_test)

255.0

In [42]:
X_test/=255

In [43]:
X_train/=255

In [44]:
np.max(X_test)

1.0

In [45]:
Y_train = pd.get_dummies(Y_train).as_matrix()

In [46]:
Y_test = pd.get_dummies(Y_test).as_matrix()

In [53]:
def ReLu(x, derivative=False):
    if(derivative==False):
        return x*(x > 0)
    else:
        return 1*(x > 0)

In [54]:
def Softmax(x):
    x -= np.max(x)
    sm = (np.exp(x).T / np.sum(np.exp(x),axis=1)).T
    return sm

In [56]:
def CreateWeights():
    ##Initialization of the Weights and the Biases with the random gaussian function with mean zeron, and variance between 1/sqtr(num_inputs_layer)
    
    ninputs = 784
    wl1 = 500 ##Number of neurons in the first layer
    wl2 = 300 ##Number of neurons in the second layer
    nclass = 10 ##Numer of the class, in this case it is the number of the digits.
    
    #layer1
    w1 = np.random.normal(0, ninputs**-0.5, [ninputs,wl1])
    b1 = np.random.normal(0, ninputs**-0.5, [1,wl1])
    
    #Layer2
    w2 = np.random.normal(0, wl1**-0.5, [wl1,wl2])
    b2 = np.random.normal(0, wl1**-0.5, [1,wl2])

    #Layer3
    w3 = np.random.normal(0, wl2**-0.5, [wl2,nclass])
    b3 = np.random.normal(0, wl2**-0.5, [1,nclass])
    
    return [w1,w2,w3,b1,b2,b3]

In [57]:
def Dropout(x, keep_prop):
    mask = np.random.binomial([np.ones_like(x)],(1-keep_prop))[0]  / (1-keep_prop)
    return x*mask

In [58]:
def predict(weights, x, keep_prop=0):
    
    w1,w2,w3,b1,b2,b3  = weights 
    
    #1-Hidden Layer
    first = ReLu(x@w1+b1)
    first = Dropout(first, keep_prop)

    #2-Hidden Layer
    second = ReLu(first@w2+b2)
    second = Dropout(second, keep_prop)
    
    #Output Layer
    return [first, second, Softmax(second@w3+b3)]

In [59]:
def accuracy(output, y):
    hit = 0
    output = np.argmax(output, axis=1)
    y = np.argmax(y, axis=1)
    for y in zip(output, y):
        if(y[0]==y[1]):
            hit += 1

    p = (hit*100)/output.shape[0]
    return p

In [60]:
def log2(x):
    if(x!=0):
        return np.log(x)
    else:
        return -np.inf
    
def log(y):
    return [[log2(nx) for nx in x]for x in y]

def cost(Y_predict, Y_right):
    
    Loss = -np.mean((np.nan_to_num(Y_right*log(Y_predict)) + np.nan_to_num((1-Y_right)*log(1-Y_predict))),keepdims=True)
    return Loss

In [62]:
x_train = X_train

In [63]:
x_valid = X_test

In [64]:
d_train = Y_train

In [65]:
d_valid = Y_test

In [66]:
def ADAM(weights, x, t, outputs, eta, beta1, beta2, eps, i, nabla, cache=None):
    
    w1,w2,w3,b1,b2,b3  = weights
    
    
    if(cache==None):
            vw1 = np.zeros_like(w1)
            mw1 = np.zeros_like(w1)
            
            vw2 = np.zeros_like(w2)
            mw2 = np.zeros_like(w2)
            
            vw3 = np.zeros_like(w3)
            mw3 = np.zeros_like(w3)
            
            vb1 = np.zeros_like(b1)
            mb1 = np.zeros_like(b1)
            
            vb2 = np.zeros_like(b2)
            mb2 = np.zeros_like(b2)
            
            vb3 = np.zeros_like(b3)
            mb3 = np.zeros_like(b3)
    else:
        vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3 = cache
    
    first, second, y = outputs
   
    w3_delta = (t-y)/x.shape[0]
    
    w2_error = w3_delta@w3.T

    w2_delta = w2_error * ReLu(second,derivative=True)

    w1_error = w2_delta@w2.T
    w1_delta = w1_error * ReLu(first,derivative=True)
    
    
    dw3 = (second.T@w3_delta + nabla*w3)
    mw3 = beta1*mw3 + (1-beta1)*dw3
    mt = (mw3) / (1-beta1**i)
    vw3 = beta2*vw3 + (1-beta2)*(dw3**2)
    vt = (vw3) / (1-beta2**i)
    w3 += eta * mt/(np.sqrt(vt) + eps)
    
    db3 = (w3_delta.sum(axis=0)+ nabla*b3)
    mb3 = beta1*mb3 + (1-beta1)*db3
    mt = (mb3) / (1-beta1**i)
    vb3 = beta2*vb3 + (1-beta2)*(db3**2)
    vt = (vb3) / (1-beta2**i)
    b3 += eta * mt/(np.sqrt(vt) + eps)
    
    dw2 = (first.T@w2_delta + nabla*w2)
    mw2 = beta1*mw2 + (1-beta1)*dw2
    mt = (mw2) / (1-beta1**i)
    vw2 = beta2*vw2 + (1-beta2)*(dw2**2)
    vt = (vw2) / (1-beta2**i)
    w2 += eta * mt/(np.sqrt(vt) + eps)
    
    db2 = (w2_delta.sum(axis=0) + nabla*b2)
    mb2 = beta1*mb2 + (1-beta1)*db2
    mt = (mb2) / (1-beta1**i)
    vb2 = beta2*vb2 + (1-beta2)*(db2**2)
    vt = (vb2) / (1-beta2**i)
    b2 += eta * mt/(np.sqrt(vt) + eps)
    
    dw1 = (x.T@w1_delta + nabla*w1)
    mw1 = beta1*mw1 + (1-beta1)*dw1
    mt = (mw1) / (1-beta1**i)
    vw1 = beta2*vw1 + (1-beta2)*(dw1**2)
    vt = (vw1) / (1-beta2**i)
    w1 += eta * mt/(np.sqrt(vt) + eps)
    
    db1 = (w1_delta.sum(axis=0) + nabla*b1)
    mb1 = beta1*mb1 + (1-beta1)*db1
    mt = (mb1) / (1-beta1**i)
    vb1 = beta2*vb1 + (1-beta2)*(db1**2)
    vt = (vb1) / (1-beta2**i)
    b1 += eta * mt/(np.sqrt(vt) + eps)
    
    
    weights = [w1,w2,w3,b1,b2,b3]
    cache = [vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3]
    
    return weights, cache

In [67]:
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [75]:
def run(weights, x_train, y_train, x_valid, y_valid, epochs = 10, nbatchs=25, alpha = 1e-3, decay = 0, beta1=0.9, beta2=0.999, eps=1e-8, l2 = 0.001, keep_prop = 0):
    
    pross = x_train.shape[0]*0.05
    
    history = [[],[]]
    
    index = np.arange(x_train.shape[0])
    cache = None
    print("Train data: %d" % (x_train.shape[0]))
    print("Validation data: %d \n" % (x_valid.shape[0]))
    mtime = 0
    inter_adam = 1
    
    for j in range(epochs):
        np.random.shuffle(index)
        t = 0
        iterations = round(x_train.shape[0]/nbatchs)
        prog = ""
        sacurr = 0
        sloss = 0
        sys.stdout.write("\nEpochs: %2d \ %2d \n"% (j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*nbatchs
            l = f+nbatchs
            
            if(l>(x_train.shape[0]-1)):
                l = x_train.shape[0]
                
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]

            outputs = predict(weights, x, keep_prop)
            
            loss = cost(outputs[-1], y)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
            
            
            accuracy_t = accuracy(outputs[-1], y)
            
            sacurr += accuracy_t
            sloss += loss
            
            accuracy_train = sacurr/(i+1)
            loss_train = sloss/(i+1)
            
            weights, cache = ADAM(weights, x, y, outputs, alpha, beta1, beta2, eps, inter_adam, l2, cache)
            
            t+= x.shape[0]
            
            qtd = round(t/pross)
            prog = "["
            for p in range(20):
                if(p<qtd-1):
                    prog += "="
                elif(p==qtd-1):
                    prog += ">"
                else:
                    prog += "."
            prog += "]"

            
            stime += time.time()-timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            
            sys.stdout.write("\r%5d/%5d %s ETA: %3d s - loss: %.4f  acc: %.4f" % (t, x_train.shape[0], prog, mTimeT, loss_train, accuracy_train))
            
            history[0].append([loss_train, accuracy_train])
            inter_adam += 1
        mtime = time.time()-timeIT
        alpha = alpha - (alpha*decay)
        
        outputs = predict(weights, x_valid)
        
        loss_valid = cost(outputs[-1], y_valid)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
        accuracy_valid = accuracy(outputs[-1], y_valid)
        
        sys.stdout.write("\r%5d/%5d %s ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0], prog, mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
        
    return weights, history

In [79]:
weights = CreateWeights()

alpha = 1e-3
epochs = 30
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.1, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.25)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 30 
60000/60000 [===================>] ETA:  55 s loss: 0.0731  acc: 86.4233 - lossValid: 0.0217  accValid: 96.0500 
Epochs:  2 \ 30 
60000/60000 [===================>] ETA:  56 s loss: 0.0404  acc: 92.7617 - lossValid: 0.0141  accValid: 97.5400 
Epochs:  3 \ 30 
60000/60000 [===================>] ETA:  59 s loss: 0.0336  acc: 94.0117 - lossValid: 0.0106  accValid: 98.1000 
Epochs:  4 \ 30 
33500/60000 [==========>.........] ETA:  24 s - loss: 0.0306  acc: 94.6925

KeyboardInterrupt: 

In [70]:
def SGD(weights, x, t, outputs, eta, gamma, nabla, cache=None):
    
    w1,w2,w3,b1,b2,b3  = weights
    
    
    if(cache==None):
            vw1 = np.zeros_like(w1)
            vw2 = np.zeros_like(w2)
            vw3 = np.zeros_like(w3)
            vb1 = np.zeros_like(b1)
            vb2 = np.zeros_like(b2)
            vb3 = np.zeros_like(b3)
    else:
        vw1,vw2,vw3,vb1,vb2,vb3 = cache
    
    first, second, y = outputs
   
    w3_delta = (t-y)
    
    w2_error = w3_delta@w3.T

    w2_delta = w2_error * ReLu(second,derivative=True)

    w1_error = w2_delta@w2.T
    w1_delta = w1_error * ReLu(first,derivative=True)
    
    eta = -eta/x.shape[0]
    nabla *= 2;
    
    vw3 = gamma*vw3 + eta * (second.T@w3_delta + nabla*w3)
    vb3 = gamma*vb3 + eta * w3_delta.sum(axis=0)

    vw2 = gamma*vw2 + eta * (first.T@w2_delta + nabla*w2)
    vb2 = gamma*vb2 + eta * w2_delta.sum(axis=0)

    vw1 = gamma*vw1 + eta * (x.T@w1_delta + nabla*w1)
    vb1 = gamma*vb1 + eta * w1_delta.sum(axis=0)
    
    
    w3 -= vw3
    b3 -= vb3

    w2 -= vw2
    b2 -= vb2

    w1 -= vw1
    b1 -= vb1
    
    weights = [w1,w2,w3,b1,b2,b3]
    cache = [vw1,vw2,vw3,vb1,vb2,vb3]
    
    return weights, cache

In [73]:
def run_sgd(weights, x_train, y_train, x_valid, y_valid, epochs = 10, nbatchs=25, alpha = 1e-3, decay = 0, momentum = 0, l2 = 0.001, keep_prop = 0):
    
    pross = x_train.shape[0]*0.05
    
    history = [[],[]]
    
    index = np.arange(x_train.shape[0])
    cache = None
    print("Train data: %d" % (x_train.shape[0]))
    print("Validation data: %d \n" % (x_valid.shape[0]))
    mtime = 0
    
    for j in range(epochs):
        np.random.shuffle(index)
        t = 0
        iterations = round(x_train.shape[0]/nbatchs)
        prog = ""
        sacurr = 0
        sloss = 0
        sys.stdout.write("\nEpochs: %2d \ %2d \n"% (j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*nbatchs
            l = f+nbatchs
            
            if(l>(x_train.shape[0]-1)):
                l = x_train.shape[0]
                
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]

            outputs = predict(weights, x, keep_prop)
            
            loss = cost(outputs[-1], y)
            
            
            accuracy_t = accuracy(outputs[-1], y)
            
            sacurr += accuracy_t
            sloss += loss
            
            accuracy_train = sacurr/(i+1)
            loss_train = sloss/(i+1)
            
            weights, cache = SGD(weights, x, y, outputs, alpha, momentum, l2, cache)
            
            t+= x.shape[0]
            
            qtd = round(t/pross)
            prog = "["
            for p in range(20):
                if(p<qtd-1):
                    prog += "="
                elif(p==qtd-1):
                    prog += ">"
                else:
                    prog += "."
            prog += "]"

            
            stime += time.time()-timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            
            sys.stdout.write("\r%5d/%5d %s ETA: %3d s - loss: %.4f  acc: %.4f" % (t, x_train.shape[0], prog, mTimeT, loss_train, accuracy_train))
            
            history[0].append([loss_train, accuracy_train])
        mtime = time.time()-timeIT
        alpha = alpha - (alpha*decay)
        
        outputs = predict(weights, x_valid)
        
        loss_valid = cost(outputs[-1], y_valid)
        accuracy_valid = accuracy(outputs[-1], y_valid)
        
        sys.stdout.write("\r%5d/%5d %s ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0], prog, mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
        
    return weights, history

In [77]:
weights = CreateWeights()

alpha = 1e-1
epochs = 30
nbatchs = 100
weights, history = run_sgd(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = alpha/(epochs*2), 
              momentum = 0.9, 
              l2 = 1e-3, 
              keep_prop = 0.25)

Train data: 60000
Validation data: 10000 


Epochs:  1 \ 30 
60000/60000 [===================>] ETA:  46 s loss: 0.0799  acc: 84.6100 - lossValid: 0.0201  accValid: 96.3700 
Epochs:  2 \ 30 
60000/60000 [===================>] ETA:  55 s loss: 0.0465  acc: 91.6050 - lossValid: 0.0154  accValid: 97.1300 
Epochs:  3 \ 30 
60000/60000 [===================>] ETA:  49 s loss: 0.0399  acc: 92.8833 - lossValid: 0.0127  accValid: 97.7900 
Epochs:  4 \ 30 
56600/60000 [==================>.] ETA:   2 s - loss: 0.0357  acc: 93.7438

KeyboardInterrupt: 